In [1]:
def flip_bits(n):
    bn = bin(n)
    flipped_bn = int("".join([str(1 - int(x)) for x in str(bn)[2:]]), 2)
    return flipped_bn

In [2]:
flip_bits(10)

5

In [3]:
flip_bits(20)

11

In [4]:
x = list(range(1000))
y = [flip_bits(i) for i in x]


In [7]:
list(zip(x, y))

[(0, 1),
 (1, 0),
 (2, 1),
 (3, 0),
 (4, 3),
 (5, 2),
 (6, 1),
 (7, 0),
 (8, 7),
 (9, 6),
 (10, 5),
 (11, 4),
 (12, 3),
 (13, 2),
 (14, 1),
 (15, 0),
 (16, 15),
 (17, 14),
 (18, 13),
 (19, 12),
 (20, 11),
 (21, 10),
 (22, 9),
 (23, 8),
 (24, 7),
 (25, 6),
 (26, 5),
 (27, 4),
 (28, 3),
 (29, 2),
 (30, 1),
 (31, 0),
 (32, 31),
 (33, 30),
 (34, 29),
 (35, 28),
 (36, 27),
 (37, 26),
 (38, 25),
 (39, 24),
 (40, 23),
 (41, 22),
 (42, 21),
 (43, 20),
 (44, 19),
 (45, 18),
 (46, 17),
 (47, 16),
 (48, 15),
 (49, 14),
 (50, 13),
 (51, 12),
 (52, 11),
 (53, 10),
 (54, 9),
 (55, 8),
 (56, 7),
 (57, 6),
 (58, 5),
 (59, 4),
 (60, 3),
 (61, 2),
 (62, 1),
 (63, 0),
 (64, 63),
 (65, 62),
 (66, 61),
 (67, 60),
 (68, 59),
 (69, 58),
 (70, 57),
 (71, 56),
 (72, 55),
 (73, 54),
 (74, 53),
 (75, 52),
 (76, 51),
 (77, 50),
 (78, 49),
 (79, 48),
 (80, 47),
 (81, 46),
 (82, 45),
 (83, 44),
 (84, 43),
 (85, 42),
 (86, 41),
 (87, 40),
 (88, 39),
 (89, 38),
 (90, 37),
 (91, 36),
 (92, 35),
 (93, 34),
 (94, 33),
 

In [1]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Final


# Bingo cards are square grids of CARD_SIZE x CARD_SIZE numbers.
CARD_SIZE: Final[int] = 5

# Pre-computed slices for rows, and lists of indices for columns
ROWS: Final[list[slice]] = [
    slice(i, i + CARD_SIZE) for i in range(0, CARD_SIZE * CARD_SIZE, CARD_SIZE)
]
COLS: Final[list[list[int]]] = [
    [cell + row for row in range(0, CARD_SIZE * CARD_SIZE, CARD_SIZE)]
    for cell in range(CARD_SIZE)
]

import pdb
@dataclass
class BingoCard:
    numbers: list[int]
    marked: set[int] = field(default_factory=set)
    _lines: frozenset[frozenset[int]] = field(init=False)

    def __post_init__(self) -> None:
        # collect the sets of numbers forming the horizontal and vertical lines
        # across the bingo card
        self._lines = frozenset(
            frozenset(self.numbers[row]) for row in ROWS
        ) | frozenset(frozenset(self.numbers[c] for c in col) for col in COLS)
        pdb.set_trace()

    @classmethod
    def from_text(cls, text: str) -> BingoCard:
        return cls([int(n) for n in text.split()])

    @property
    def unmarked_score(self) -> int:
        return sum(set(self.numbers) - self.marked)

    @property
    def wins(self) -> bool:
        # a bingo card wins if any of its lines is a subset of the marked numbers
        marked = self.marked
        return any(marked >= line for line in self._lines)

    def mark(self, number: int) -> int:
        """Mark a number off on the card, and return a score

        The score is 0 if there are still rows or columns with unmarked numbers.

        """
        pdb.set_trace()
        self.marked.add(number)
        if self.wins:
            return self.unmarked_score * number
        return 0


@dataclass
class BingoSubsystem:
    random_numbers: list[int]
    cards: list[BingoCard] = field(default_factory=list)
    number_index: dict[int, set[int]] = field(default_factory=dict)

    def add_card(self, card: BingoCard) -> None:
        card_index = len(self.cards)
        self.cards.append(card)
        for number in card.numbers:
            self.number_index.setdefault(number, set()).add(card_index)

    @classmethod
    def from_text(cls, text: str) -> BingoSubsystem:
        blocks = iter(text.split("\n\n"))
        game = cls([int(n) for n in next(blocks).split(",")])
        for block in blocks:
            game.add_card(BingoCard.from_text(block))
        return game

    def find_winning_score(self):
        empty = frozenset()
        for number in self.random_numbers:
            for card_index in self.number_index.get(number, empty):
                if score := self.cards[card_index].mark(number):
                    return score
        return 0


test_input = """\
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""

assert BingoSubsystem.from_text(test_input).find_winning_score() == 4512

--Return--
None
> /var/folders/yr/rg2q2zm56tb4l9wjgmqk0zcr0000gp/T/ipykernel_34689/2162027253.py(31)__post_init__()
     29             frozenset(self.numbers[row]) for row in ROWS
     30         ) | frozenset(frozenset(self.numbers[c] for c in col) for col in COLS)
---> 31         pdb.set_trace()
     32 
     33     @classmethod



In [26]:
# get execution time for the code above
with open("./2021/inputs/day_4_sample.txt", "r") as f:
    test_input = f.read()
import time
start = time.time()
_ = BingoSubsystem.from_text(test_input).find_winning_score()
end = time.time()
print(end - start)


0.0006239414215087891


In [24]:
import requests

class AOCDay:
    def __init__(self, filepath):
        self.set_input(
            filepath
        )

    def set_input(self, filepath=None):
        if filepath is None:
            headers = {
                'cookie': 'session=53616c7465645f5f675aea7adfc467a0be6ab4893603804db460ec1a78c1027ba3b4b1e57038e68251ae350f72853029',
            }
            self.raw_input = requests.get(f"https://adventofcode.com/2021/day/{self.day}/input", headers=headers).content.decode('utf-8').strip()
        else:
            with open(filepath) as f:
                self.raw_input = f.read().strip()
        
        self.input_lines = [l.strip() for l in self.raw_input.split('\n')]

import pdb
import copy as cp

class Day4(AOCDay):
    def __init__(self, filepath=None):
        self.day = 4
        super().__init__(filepath)
        self.draw_list = list(map(int, self.input_lines[0].split(",")))
        self.boards = [[list(map(int, y.split())) for y in x.split("\n")] for x in self.raw_input.split("\n\n")[1:]]

    def calc_boards_score(self):
        boards_score = []
        list_score = lambda l: int(max(l, key=lambda x: self.draw_list.index(x))) if not len(set(l).difference(set(self.draw_list))) else -1
        for i, b in enumerate(self.boards):
            max_score = (0, 1000)
            b_t = list(map(list, zip(*b)))
            for r in b:
                score = list_score(r)
                score_index = self.draw_list.index(score) if score >= 0 else 1000
                if score >= 0 and score_index < max_score[1]:
                    max_score = (score, score_index, i)
            for r in b_t:
                score = list_score(r)
                score_index = self.draw_list.index(score) if score >= 0 else 1000
                if score >= 0 and score_index < max_score[1]:
                    max_score = (score, score_index, i)
            boards_score.append(max_score)
        return boards_score

    def solve_part_1(self):
        best_board_score = min(self.calc_boards_score(), key=lambda x: x[1])
        return best_board_score[0]*sum(set(sum(self.boards[best_board_score[2]], [])).difference(set(self.draw_list[:best_board_score[1]+1])))
        

    def solve_part_2(self):
        best_board_score = max(self.calc_boards_score(), key=lambda x: x[1])
        return best_board_score[0]*sum(set(sum(self.boards[best_board_score[2]], [])).difference(set(self.draw_list[:best_board_score[1]+1])))

In [29]:
# get execution time for the code above
import time
day = Day4()
start = time.time()
_ = day.solve_part_1()
end = time.time()
print(end - start)
# day = Day4()

0.014241933822631836


In [32]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Final

CARD_SIZE = 2
# Bingo cards are square grids of CARD_SIZE x CARD_SIZE numbers.
CARD_SIZE: Final[int] = 5

# Pre-computed slices for rows, and lists of indices for columns
ROWS: Final[list[slice]] = [
    slice(i, i + CARD_SIZE) for i in range(0, CARD_SIZE * CARD_SIZE, CARD_SIZE)
]
COLS: Final[list[list[int]]] = [
    [cell + row for row in range(0, CARD_SIZE * CARD_SIZE, CARD_SIZE)]
    for cell in range(CARD_SIZE)

SyntaxError: unexpected EOF while parsing (4037067695.py, line 15)

In [33]:
print("a")

a


In [34]:
CARD_SIZE

5

In [35]:
ROWS

[slice(0, 5, None),
 slice(5, 10, None),
 slice(10, 15, None),
 slice(15, 20, None),
 slice(20, 25, None)]

In [36]:
COLS

[[0, 5, 10, 15, 20],
 [1, 6, 11, 16, 21],
 [2, 7, 12, 17, 22],
 [3, 8, 13, 18, 23],
 [4, 9, 14, 19, 24]]